In [ ]:
import sys
import pandas as pd
import numpy as np

from utils import pre_all
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
print(f"파이썬 버전 : {sys.version}")
print(f"pandas 버전 : {pd.__version__}")
print(f"numpy 버전 : {np.__version__}")

In [ ]:
# train_df = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)']]
# test_df = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/test.csv')[['ID','timestamp']]
train_df = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/train.csv')
test_df = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/test.csv')

In [ ]:
train_df['item_id'] = train_df.ID.str[0:6]
test_df['item_id'] = test_df.ID.str[0:6]

In [ ]:
train, test = pre_all(train_df, test_df)

In [ ]:
train

In [ ]:
test

In [ ]:
# make static_features_df
static_features_df = train.loc[:, ['item_id', 'item', 'corporation', 'location']]
static_features_df = static_features_df.drop_duplicates()
print(static_features_df.head())

In [ ]:
train.drop(columns=['ID', 'item', 'corporation', 'location'], inplace=True)
test.drop(columns=['ID'], inplace=True)

In [ ]:
print(train.head())
print(test.head())

In [ ]:
# create time-varying covariates
# Create a pivot table to transform the DataFrame
pivot_table = train.pivot_table(index='timestamp', columns='item_id', values='price', aggfunc='first')
pivot_table.columns = [c + "_prev_price" for c in pivot_table.columns]

In [ ]:
pivot_table.columns

In [ ]:
pivot_table.shape

In [ ]:
pivot_table['prev_date'] = pivot_table.index

In [ ]:
pivot_table

In [ ]:
# left join pivot_table with train_data
train['prev_date'] = train['timestamp'] - pd.Timedelta(days=1)
train = pd.merge(train, pivot_table, on=["prev_date"], how="left")
train

In [ ]:
# drop 2019-01-01 data since past price covariate does not exist
# Define the threshold date
threshold_date = pd.to_datetime('2019-01-01')

# Drop rows where timestamp is less than the threshold date
train = train[train['timestamp'] > threshold_date]
train

In [ ]:
train_data

In [ ]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train,
    id_column="item_id",
    timestamp_column="timestamp",
    static_features_df=static_features_df,
)
print(train_data.head())
print(train_data.static_features.head())

In [ ]:
train_data.columns

In [ ]:
train_data.drop(columns=['prev_date'], inplace=True)

In [ ]:
train_data.head()
# train_data.index[:][1] < pd.to_datetime('2019-01-01')
train_data.index.get_level_values('item_id')

In [ ]:
train_data.shape

In [95]:
# create own validation set (march 4~31 of 2019, 2020, 2021, 2022)
march_2019_threshold = pd.to_datetime('2019-03-31')
val_set_1 = train_data[train_data.index.get_level_values('timestamp') <= march_2019_threshold]

march_2020_threshold = pd.to_datetime('2020-03-31')
val_set_2 = train_data[(train_data.index.get_level_values('timestamp') > march_2019_threshold) & (train_data.index.get_level_values('timestamp') <= march_2020_threshold)]

march_2021_threshold = pd.to_datetime('2021-03-31')
val_set_3 = train_data[(train_data.index.get_level_values('timestamp') > march_2020_threshold) & (train_data.index.get_level_values('timestamp') <= march_2021_threshold)]

march_2022_threshold = pd.to_datetime('2022-03-31')
val_set_4 = train_data[(train_data.index.get_level_values('timestamp') > march_2021_threshold) & (train_data.index.get_level_values('timestamp') <= march_2022_threshold)]

my_validation_dataset = pd.concat([val_set_1, val_set_2, val_set_3, val_set_4], axis=0)
my_validation_dataset


,,supply,price,year,month,day,week_day,year_month,week,week_num,holiday,...,TG_A_J_prev_price,TG_A_S_prev_price,TG_B_J_prev_price,TG_B_S_prev_price,TG_C_J_prev_price,TG_C_S_prev_price,TG_D_J_prev_price,TG_D_S_prev_price,TG_E_J_prev_price,TG_E_S_prev_price
item_id,timestamp,,,,,,,,,,,,,,,,,,,,,
RD_F_J,2019-01-02,0.0,0.0,2019,1,2,2,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BC_A_J,2019-01-02,0.0,0.0,2019,1,2,2,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RD_E_S,2019-01-02,0.0,0.0,2019,1,2,2,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TG_A_S,2019-01-02,0.0,0.0,2019,1,2,2,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BC_E_S,2019-01-02,0.0,0.0,2019,1,2,2,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CB_D_J,2022-03-31,7664.0,913.0,2022,3,31,3,41,13,171,0,...,6249.0,5330.0,8639.0,5103.0,5627.0,5644.0,5708.0,4322.0,0.0,4770.0
BC_B_S,2022-03-31,1344.0,3500.0,2022,3,31,3,41,13,171,0,...,6249.0,5330.0,8639.0,5103.0,5627.0,5644.0,5708.0,4322.0,0.0,4770.0
BC_E_S,2022-03-31,264.0,3125.0,2022,3,31,3,41,13,171,0,...,6249.0,5330.0,8639.0,5103.0,5627.0,5644.0,5708.0,4322.0,0.0,4770.0


In [ ]:
# configure target and known, past covariates in predictor
predictor = TimeSeriesPredictor( 
    prediction_length=28,
    target="price",
    known_covariates_names=["year", "month", "day", "week_day", "year_month", "week", "week_num", "holiday"] # supply and x_prev_price columns will automatically interpreted as past covariates
    eval_metric="RMSE",
)

# seed 고정
predictor.fit(train_data,
              presets="high_quality",
              random_seed=42,
              tuning_data=my_validation_dataset,
              # num_val_windows=3,
              )

In [ ]:
predictor.refit_full()

In [ ]:
# seed 고정
pred = predictor.predict(data, random_seed=42, )
pred

In [ ]:
# # The test score is computed using the last
# # prediction_length=48 timesteps of each time series in test_data
# predictor.leaderboard(test_data)

In [ ]:
import matplotlib.pyplot as plt

# TimeSeriesDataFrame can also be loaded directly from a file
# test_data = TimeSeriesDataFrame.from_path("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/test.csv")

plt.figure(figsize=(20, 3))

# item_id = "H1"
item_id = "TG_A_J"
y_past = data.loc[item_id]["price(원/kg)"]
y_pred = pred.loc[item_id]
# y_test = test_data.loc[item_id]["price(원/kg)"][-48:]

plt.plot(y_past[-200:], label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")
# plt.plot(y_test, label="Future time series values")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
)
plt.legend();

In [ ]:
submission = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/dacon_submission.csv', index=False)
submission